In [123]:
import pandas as pd
import numpy as np
import requests
import io
from operator import imod
from bs4 import BeautifulSoup

In [ ]:
#Part 1

In [357]:

#1. Clean/wrangle/combine these data in to one pandas DataFrame with exactly four columns: (1) country, (2) year, (3) life expectency, (4) happiness score. 
# When combining, keep only country and year combinations that are in both datasets.

url = "https://github.com/stat386-fall2022/homework-data/blob/main/Gapminder/hapiscore_whr.csv"

download = requests.get(url).content

happy = pd.read_html(download)[0].drop("Unnamed: 0", axis=1)

happy = pd.melt(happy, id_vars = 'country', var_name = 'year', value_name= 'happiness')



url1 = "https://github.com/stat386-fall2022/homework-data/blob/main/Gapminder/life_expectancy_years.csv"

download1 = requests.get(url1).content

life = pd.read_html(download1)[0].drop("Unnamed: 0", axis=1)

life = pd.melt(life, id_vars = 'country', var_name = 'year', value_name= 'life_expectancy')



df = pd.merge(happy, life, on=["country", "year"])


In [355]:
#2. How many rows are in the final dataset?

df

,country,year,happiness,life_expectancy
0,Afghanistan,2004,NaN,57.6
1,Angola,2004,NaN,56.4
2,Albania,2004,NaN,76.2
3,United Arab Emirates,2004,NaN,69.2
4,Argentina,2004,NaN,75.3
...,...,...,...,...
2766,Vietnam,2020,54.9,74.7
2767,Yemen,2020,42.0,67.9
2768,South Africa,2020,51.9,65.6
2769,Zambia,2020,37.6,63.8


In [118]:
#3. How many missing values are in each of the four columns?

df.isna().sum()

country              0
year                 0
happiness          704
life_expectancy      0
dtype: int64

In [119]:
#4. Ignoring any missing values, what is the correlation coefficient between life expectancy and happiness score (round to 3 decimal places)?

df.dropna().corr().round(decimals=3)

,happiness,life_expectancy
happiness,1.000,0.737
life_expectancy,0.737,1.000


In [121]:
#5. Ignoring any missing values, which country has the highest mean life expectency? Report the country and the value of the mean (rounded to 2 decimal places).

print(df.dropna().groupby('country')['life_expectancy'].mean().idxmax())
print(df.dropna().groupby('country')['life_expectancy'].mean().max().round(decimals=2))

Japan
83.92


In [122]:
#6. Ignoring any missing values, which country has the hightest mean happiness score? Report the country and the value of the mean (rounded to 2 decimal places).

print(df.dropna().groupby('country')['happiness'].mean().idxmax())
print(df.dropna().groupby('country')['happiness'].mean().max().round(decimals=2))

Denmark
76.78


In [359]:
#7. Now fill in the missing values for the happiness score with the country specific mean. For example, the data for Afghanistan are shown below:

def fill_mean(group):
    return group.fillna(group.mean())

df = df.groupby('country').apply(fill_mean)

df

/var/folders/p5/jvc140rx7rd0hd5zsvsd5px00000gn/T/ipykernel_14619/681074203.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return group.fillna(group.mean())


,country,year,happiness,life_expectancy
0,Afghanistan,2004,35.015385,57.6
1,Angola,2004,44.225000,56.4
2,Albania,2004,50.321429,76.2
3,United Arab Emirates,2004,67.850000,69.2
4,Argentina,2004,62.887500,75.3
...,...,...,...,...
2766,Vietnam,2020,54.900000,74.7
2767,Yemen,2020,42.000000,67.9
2768,South Africa,2020,51.900000,65.6
2769,Zambia,2020,37.600000,63.8


In [361]:
#8. After filling the missing values, what are the countries with the highest and lowest median happiness scores? Report both the country and the median rounded to 2 decimal places.

#Higest
print(df.groupby('country')['happiness'].median().idxmax())
print(df.groupby('country')['happiness'].median().max().round(decimals=2))

Denmark
76.5


In [362]:
#8.
#Lowest
print(df.groupby('country')['happiness'].median().idxmin())
print(df.groupby('country')['happiness'].median().min().round(decimals=2))

South Sudan
34.02


In [ ]:
#Part 2

In [363]:

#1. Compile a dataset of all the NBA players available at these websites. Your dataset should include the following variables:

#function
def in_players(initial, sec, elem, list, alph):
    
    url = f"https://www.basketball-reference.com/players/{alph}/"
    r = requests.get(url)
    soup = BeautifulSoup(r.text) 

    element = soup.find_all(initial, {sec: elem})

    for e in element:
        list.append(e.get_text())


#Lists
player = []
from_year = []
to_year = []
pos = []
height = []
weight = []
birth_date = []
college = []

alphabet = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","y","z"]



for letter in alphabet:
    in_players("th", "class", "left", player, letter)
    in_players("td", "data-stat", "year_min", from_year, letter)
    in_players("td", "data-stat", "year_max", to_year, letter)
    in_players("td", "data-stat", "pos", pos, letter)
    in_players("td", "data-stat", "height", height, letter)
    in_players("td", "data-stat", "weight", weight, letter)
    in_players("td", "data-stat", "birth_date", birth_date, letter)
    in_players("td", "data-stat", "colleges", college, letter)
    

#dataframe
all_players = pd.DataFrame({'Player': player, 'From': from_year, 'To': to_year, 'Pos': pos, 'Ht': height, 
                            'Wt': weight, 'BirthDate': birth_date, 'Colleges': college})

#changing values to numeric
all_players['From'] = pd.to_numeric(all_players['From']) 
all_players['To'] = pd.to_numeric(all_players['To']) 
all_players['Wt'] = pd.to_numeric(all_players['Wt']) 

In [364]:
all_players

,Player,From,To,Pos,Ht,Wt,BirthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"
...,...,...,...,...,...,...,...,...
5077,Ante Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",
5078,Jim Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University
5079,Bill Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne
5080,Ivica Zubac,2017,2023,C,7-0,240.0,"March 18, 1997",


In [366]:
#2. Further clean/transform the scraped data by casting "From", "To", and "Wt" into numeric data. Also compute the following variables:

#years_played = number of years played, computed as (To - From + 1). Don't forget to include the the +1 so that a player who has the same "From" and "To" year will be counted as playing 1 year.
all_players['years_played'] = (all_players['To'] - all_players['From'] + 1)


#Ht_in = Height of the player in inches (recall that there are 12 inches in a foot)
all_players['Ht_in'] = (pd.to_numeric(all_players['Ht'].str[0:1])*12) + pd.to_numeric(all_players['Ht'].str[2:])


#HoF = indicator that is True or 1 if the player is in the Hall of Fame and False or 0 otherwise (note that the name has an * at the end if the player is in the hall of fame)
all_players['HoF'] = np.where(all_players['Player'].str.find('*') == -1, False, True)


#start_age = Age at entrance into the NBA. This can be computed by subtracting the year at start (the "From" year) from the birth year -- don't worry about adjusting for 
# birth month or day. Set this value to "missing" if the Birth Date or "From" year is missing.
all_players['start_age'] = (all_players['From'] - pd.to_numeric(all_players['BirthDate'].str[-4:])).fillna('missing')

In [367]:
all_players

,Player,From,To,Pos,Ht,Wt,BirthDate,Colleges,years_played,Ht_in,HoF,start_age
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke,5,82,False,23.0
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State,10,81,False,23.0
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA,20,86,True,23.0
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU,11,73,False,22.0
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State",6,78,False,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5077,Ante Žižić,2018,2020,F-C,6-10,266.0,"January 4, 1997",,3,82,False,21.0
5078,Jim Zoet,1983,1983,C,7-1,240.0,"December 20, 1953",Kent State University,1,85,False,30.0
5079,Bill Zopf,1971,1971,G,6-1,170.0,"June 7, 1948",Duquesne,1,73,False,23.0
5080,Ivica Zubac,2017,2023,C,7-0,240.0,"March 18, 1997",,7,84,False,20.0


In [368]:
#3. Answer the following questions about the cleaned data:

#How many players are in the Hall of Fame?
all_players['HoF'].value_counts()

False    4918
True      164
Name: HoF, dtype: int64

In [369]:
#b. What is the latest "To" year of players in the Hall of Fame?
max_hof_year = all_players.loc[all_players['HoF'] == True]['To'].max()
max_hof_year

2018

In [370]:
# Which player or players in the Hall of Fame played until the year you found in part (b)?

hof = all_players.loc[all_players['HoF'] == True]
not_hof = all_players.loc[all_players['HoF'] == False]

hof.loc[hof['To'] == max_hof_year]

,Player,From,To,Pos,Ht,Wt,BirthDate,Colleges,years_played,Ht_in,HoF,start_age
1587,Manu Ginóbili*,2003,2018,G,6-6,205.0,"July 28, 1977",,16,78,True,26.0


In [371]:
#Fill in the table below with corresponding means rounded to 2 decimal places.

#Hall of Famers
hof[['Ht_in', 'Wt', 'years_played', 'start_age']].mean().round(decimals=2)

Ht_in            78.23
Wt              208.23
years_played     12.51
start_age        23.68
dtype: float64

In [372]:
#Non Hall of Famers
f_not_hof = not_hof[['Ht_in', 'Wt', 'years_played', 'start_age']]

f_not_hof[f_not_hof.start_age != 'missing'].mean().round(decimals=2)

Ht_in            78.00
Wt              209.41
years_played      4.99
start_age        23.85
dtype: float64